In [1]:
# Задание на повторение материала предыдущего семестра

In [3]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVR, SVC
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.cluster import KMeans

from sklearn.metrics import mean_squared_error, f1_score, silhouette_score

In [4]:
# Генерируем уникальный seed
my_code = "Osmanov"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [6]:
# Данные загружены отсюда: https://www.kaggle.com/dwdkills/russian-demography
# Читаем данные из файла
example_data = pd.read_csv("datasets/russian_demography.csv")

In [7]:
# "year" - год (1990-2017)
# "region" - название региона
# "npg" - естественный прирост населения на 1000 человек
# "birth_rate" - количество рождений на 1000 человек
# "death_rate" - количество смертей на 1000 человек
# "gdw" - коэффициент демографической нагрузки на 100 человек (Отношение числа нетрудоспособных к числу трудоспособных).
# "urbanization" - процент городского населения

example_data.head()

,year,region,npg,birth_rate,death_rate,gdw,urbanization
0,1990,Republic of Adygea,1.9,14.2,12.3,84.66,52.42
1,1990,Altai Krai,1.8,12.9,11.1,80.24,58.07
2,1990,Amur Oblast,7.6,16.2,8.6,69.55,68.37
3,1990,Arkhangelsk Oblast,3.7,13.5,9.8,73.26,73.63
4,1990,Astrakhan Oblast,4.7,15.1,10.4,77.05,68.01


In [8]:
# Так как список регионов меняется от года к году, в данных есть строки без значений. Удалим их
example_data.dropna(inplace=True)

In [9]:
example_data.shape

(2315, 7)

In [10]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(example_data))
print(val_test_size)

463


In [11]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(example_data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

1389 463 463


In [12]:
train.head()

,year,region,npg,birth_rate,death_rate,gdw,urbanization
820,1999,Tuva Republic,2.5,16.0,13.5,76.96,49.20
1589,2008,Saint Petersburg,-4.2,10.4,14.0,58.28,100.00
2169,2015,Novgorod Oblast,-5.6,11.9,17.5,80.78,70.90
224,1992,Komi Republic,2.0,11.4,9.4,62.94,75.73
2174,2015,Penza Oblast,-4.1,10.7,14.8,75.99,68.10


In [13]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
columns_to_scale = ['year', 'npg', 'birth_rate', 'death_rate', 'gdw', 'urbanization']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['year', 'npg', 'birth_rate', 'death_rate',
                                  'gdw', 'urbanization'])])

In [14]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [15]:
# Устанавливаем названия столбцов
column_names = columns_to_scale + ['region']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [16]:
sc_val

,year,npg,birth_rate,death_rate,gdw,urbanization,region
0,0.259259,0.293827,0.108225,0.431925,0.338668,0.814718,Sakhalin Oblast
1,0.740741,0.246914,0.17316,0.591549,0.478344,0.565274,Penza Oblast
2,0.296296,0.434568,0.268398,0.338028,0.688468,0.215922,Republic of Kalmykia
3,0.222222,0.283951,0.125541,0.469484,0.539398,0.550609,Tomsk Oblast
4,0.962963,0.385185,0.311688,0.478873,0.695251,0.474925,Orenburg Oblast
...,...,...,...,...,...,...,...
458,0.444444,0.254321,0.164502,0.568075,0.421812,0.672254,Novosibirsk Oblast
459,0.37037,0.279012,0.17316,0.530516,0.411028,0.560822,Amur Oblast
460,0.62963,0.25679,0.190476,0.624413,0.323361,0.693597,Republic of Karelia
461,0.296296,0.518519,0.47619,0.403756,0.708993,0.328401,Tuva Republic


In [17]:
# Вспоминаем алгоритмы решения задачи регрессии
r_models = []

# Линейная регрессия
# Для использования регуляризации, вместо LinearRegression используем Lasso, Ridge или ElasticNet
# Параметр alpha - коэффициент регуляризации для Lasso и Ridge, по умолчанию равен 1
# Для ElasticNet, если регуляризация иммет вид a*L1+b*L2, то
# параметр alpha = a + b, по умолчанию равен 1
# параметр l1_ratio = a / (a + b), по умолчанию равен 0.5
r_models.append(LinearRegression())
r_models.append(Lasso(alpha=1.0))
r_models.append(Ridge(alpha=1.0))
r_models.append(ElasticNet(alpha=1.0, l1_ratio=0.5))

# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
r_models.append(KNeighborsRegressor(n_neighbors=5))
r_models.append(KNeighborsRegressor(n_neighbors=10))
r_models.append(KNeighborsRegressor(n_neighbors=15))

# Метод опорных векторов
# Параметр kernel опредеяет вид ядра преобразования
r_models.append(SVR(kernel='linear'))
r_models.append(SVR(kernel='poly'))
r_models.append(SVR(kernel='rbf'))
r_models.append(SVR(kernel='sigmoid'))

# Деревья решений
# Параметр criterion - критерий качества ветвления: 'mse', 'friedman_mse', 'mae', 'poisson'
r_models.append(DecisionTreeRegressor(criterion='squared_error'))
r_models.append(DecisionTreeRegressor(criterion='friedman_mse'))
r_models.append(DecisionTreeRegressor(criterion='absolute_error'))

In [18]:
r_models

[LinearRegression(),
 Lasso(),
 Ridge(),
 ElasticNet(),
 KNeighborsRegressor(),
 KNeighborsRegressor(n_neighbors=10),
 KNeighborsRegressor(n_neighbors=15),
 SVR(kernel='linear'),
 SVR(kernel='poly'),
 SVR(),
 SVR(kernel='sigmoid'),
 DecisionTreeRegressor(),
 DecisionTreeRegressor(criterion='friedman_mse'),
 DecisionTreeRegressor(criterion='absolute_error')]

In [19]:
# Выделим предикторы и зависимую переменную
x_labels = column_names[0:-2]
y_labels = ['urbanization']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [20]:
x_labels

['year', 'npg', 'birth_rate', 'death_rate', 'gdw']

In [21]:
y_labels

['urbanization']

In [22]:
# Обучаем модели
for model in r_models:
    print(model)
    model.fit(x_train, y_train)

LinearRegression()
Lasso()
Ridge()
ElasticNet()
KNeighborsRegressor()
KNeighborsRegressor(n_neighbors=10)
KNeighborsRegressor(n_neighbors=15)
SVR(kernel='linear')
SVR(kernel='poly')
SVR()
SVR(kernel='sigmoid')
DecisionTreeRegressor()
DecisionTreeRegressor(criterion='friedman_mse')
DecisionTreeRegressor(criterion='absolute_error')


In [23]:
# Оценииваем качество работы моделей на валидационной выборке.
mses = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(model, '\t', mse)

LinearRegression() 	 0.018940049528400227
Lasso() 	 0.02727575830955416
Ridge() 	 0.0187346936205259
ElasticNet() 	 0.02727575830955416
KNeighborsRegressor() 	 0.010815765305486705
KNeighborsRegressor(n_neighbors=10) 	 0.011793272562087452
KNeighborsRegressor(n_neighbors=15) 	 0.012330080595645438
SVR(kernel='linear') 	 0.018745143612210847
SVR(kernel='poly') 	 0.0123039314137817
SVR() 	 0.010695626908387448
SVR(kernel='sigmoid') 	 29.332296244168187
DecisionTreeRegressor() 	 0.02091569234216171
DecisionTreeRegressor(criterion='friedman_mse') 	 0.020951103813144727
DecisionTreeRegressor(criterion='absolute_error') 	 0.02000074644981771


In [24]:
# Выбираем лучшую модель
i_min = mses.index(min(mses))
best_r_model = r_models[i_min]
best_r_model

SVR()

In [25]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

0.012442388560291303


In [26]:
# Вспоминаем алгоритмы решения задачи классификации
c_models = []

# Логистическая регрессия
# Параметр penalty - тип регуляризации: 'l1', 'l2', 'elasticnet', 'none'}, по умолчанию 'l2'
# Для некоторых типов регуляризации доступны не все алгоритмы (параметр solver)
# Для elasticnet регуляризации необходимо уазывать параметр l1_ratio (0 - l2, 1 - l1)
c_models.append(LogisticRegression(penalty='none', solver='saga', max_iter=500))
c_models.append(LogisticRegression(penalty='l1', solver='saga', max_iter=500))
c_models.append(LogisticRegression(penalty='l2', solver='saga'))
c_models.append(LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga'))
c_models.append(LogisticRegression())

# Наивный байесовский классификатор
# Параметр alpha - параметр сглаживания, по умолчанию равен 1 (сглаживание Лапласа)
c_models.append(MultinomialNB(alpha=0.0))
c_models.append(MultinomialNB(alpha=0.5))
c_models.append(MultinomialNB(alpha=1.0))

# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
c_models.append(KNeighborsClassifier(n_neighbors=5))
c_models.append(KNeighborsClassifier(n_neighbors=10))
c_models.append(KNeighborsClassifier(n_neighbors=15))

# Метод опорных векторов
# Параметр kernel опредеяет вид ядра преобразования
c_models.append(SVC(kernel='linear'))
c_models.append(SVC(kernel='poly'))
c_models.append(SVC(kernel='rbf'))
c_models.append(SVC(kernel='sigmoid'))

# Деревья решений
# Параметр criterion - критерий качества ветвления: 'gini', 'entropy'
# Параметр splitter - стартегия ветвления: 'best', 'random'
c_models.append(DecisionTreeClassifier(criterion='gini', splitter='best'))
c_models.append(DecisionTreeClassifier(criterion='gini', splitter='random'))
c_models.append(DecisionTreeClassifier(criterion='entropy', splitter='best'))
c_models.append(DecisionTreeClassifier(criterion='entropy', splitter='random'))

In [27]:
c_models

[LogisticRegression(max_iter=500, penalty='none', solver='saga'),
 LogisticRegression(max_iter=500, penalty='l1', solver='saga'),
 LogisticRegression(solver='saga'),
 LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga'),
 LogisticRegression(),
 MultinomialNB(alpha=0.0),
 MultinomialNB(alpha=0.5),
 MultinomialNB(),
 KNeighborsClassifier(),
 KNeighborsClassifier(n_neighbors=10),
 KNeighborsClassifier(n_neighbors=15),
 SVC(kernel='linear'),
 SVC(kernel='poly'),
 SVC(),
 SVC(kernel='sigmoid'),
 DecisionTreeClassifier(),
 DecisionTreeClassifier(splitter='random'),
 DecisionTreeClassifier(criterion='entropy'),
 DecisionTreeClassifier(criterion='entropy', splitter='random')]

In [28]:
# Выделим предикторы и метки классов
x_labels = column_names[0:-1]
y_labels = ['region']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [29]:
x_train

,year,npg,birth_rate,death_rate,gdw,urbanization
0,0.333333,0.449383,0.424242,0.478873,0.68638,0.334817
1,0.666667,0.283951,0.181818,0.502347,0.361454,1.0
2,0.925926,0.249383,0.246753,0.666667,0.752827,0.61896
3,0.074074,0.437037,0.225108,0.286385,0.442512,0.682205
4,0.925926,0.28642,0.194805,0.539906,0.669508,0.582297
...,...,...,...,...,...,...
1384,0.444444,0.264198,0.147186,0.530516,0.543225,0.422941
1385,0.296296,0.212346,0.012987,0.483568,0.539746,1.0
1386,0.518519,0.283951,0.194805,0.549296,0.366151,0.595391
1387,0.740741,0.945679,0.904762,0.056338,0.500087,0.249705


In [30]:
# Обучаем модели
for model in c_models:
    print(model)
    model.fit(x_train, y_train)

LogisticRegression(max_iter=500, penalty='none', solver='saga')


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


LogisticRegression(max_iter=500, penalty='l1', solver='saga')
LogisticRegression(solver='saga')
LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga')
LogisticRegression()
MultinomialNB(alpha=0.0)
MultinomialNB(alpha=0.5)
MultinomialNB()


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/naive_bayes.py:557: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  % _ALPHA_MIN


KNeighborsClassifier()
KNeighborsClassifier(n_neighbors=10)
KNeighborsClassifier(n_neighbors=15)
SVC(kernel='linear')
SVC(kernel='poly')
SVC()
SVC(kernel='sigmoid')
DecisionTreeClassifier()
DecisionTreeClassifier(splitter='random')
DecisionTreeClassifier(criterion='entropy')
DecisionTreeClassifier(criterion='entropy', splitter='random')


In [31]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model in c_models:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    f1s.append(f1)
    print(model, '\t', f1)

LogisticRegression(max_iter=500, penalty='none', solver='saga') 	 0.3796130502167596
LogisticRegression(max_iter=500, penalty='l1', solver='saga') 	 0.14777003954995457
LogisticRegression(solver='saga') 	 0.06256800451572904
LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga') 	 0.05569351460796611
LogisticRegression() 	 0.06256800451572904
MultinomialNB(alpha=0.0) 	 0.010506219720706613
MultinomialNB(alpha=0.5) 	 0.009984009062617559
MultinomialNB() 	 0.009298721954770007
KNeighborsClassifier() 	 0.35000217731691957
KNeighborsClassifier(n_neighbors=10) 	 0.24223711455642422
KNeighborsClassifier(n_neighbors=15) 	 0.21351376543922132
SVC(kernel='linear') 	 0.06279212341634302
SVC(kernel='poly') 	 0.6950084198136676
SVC() 	 0.19111369573451156
SVC(kernel='sigmoid') 	 8.342680225066974e-05
DecisionTreeClassifier() 	 0.6023019457144726
DecisionTreeClassifier(splitter='random') 	 0.6133465846254544
DecisionTreeClassifier(criterion='entropy') 	 0.6202667591790018
DecisionTre

In [32]:
# Выбираем лучшую модель
i_max = f1s.index(max(f1s))
best_c_model = c_models[i_max]
best_c_model

SVC(kernel='poly')

In [33]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_c_model.predict(x_test)
f1 = f1_score(y_test, test_pred, average='weighted')
print(f1)

0.7004938165395607


In [34]:
# Вспоминаем алгоритм решения задачи кластеризации - метод k-средних
# Параметр n_clusters - количество кластеров, по умолчанию равен 8
k_models = []
k_models.append(KMeans(n_clusters=5))
k_models.append(KMeans(n_clusters=8))
k_models.append(KMeans(n_clusters=20))
k_models.append(KMeans(n_clusters=50))

In [35]:
# Выделим используемые параметры
x_labels = column_names[0:-1]
x = pd.concat([sc_train[x_labels], sc_val[x_labels], sc_test[x_labels]])
x

,year,npg,birth_rate,death_rate,gdw,urbanization
0,0.333333,0.449383,0.424242,0.478873,0.68638,0.334817
1,0.666667,0.283951,0.181818,0.502347,0.361454,1.0
2,0.925926,0.249383,0.246753,0.666667,0.752827,0.61896
3,0.074074,0.437037,0.225108,0.286385,0.442512,0.682205
4,0.925926,0.28642,0.194805,0.539906,0.669508,0.582297
...,...,...,...,...,...,...
458,0.296296,0.175309,0.056277,0.600939,0.801009,0.494959
459,0.962963,0.419753,0.341991,0.446009,0.702383,0.591463
460,0.592593,0.271605,0.181818,0.558685,0.522004,0.363624
461,0.666667,0.274074,0.190476,0.568075,0.427727,0.662171


In [36]:
# Произведем кластеризацию
for model in k_models:
    model.fit(x)

In [37]:
# Оценим качество результата
sils = []
for model in k_models:
    cluster_labels = model.predict(x)
    s = silhouette_score(x, cluster_labels)
    sils.append(s)
    print(s)

0.2950374076127764
0.2546821749741407
0.23583991800146994
0.23182663006814397


In [38]:
# Выбираем лучшую модель
i_max = sils.index(max(sils))
best_k_model = k_models[i_max]
print(best_k_model)
print(sils[i_max])

KMeans(n_clusters=5)
0.2950374076127764


In [39]:
# Задание №1 - анализ моделей для задачи регрессии
# Общий список моделей
r_models = [
    LinearRegression(),
    Lasso(alpha=1.0),
    Lasso(alpha=0.5),
    Ridge(alpha=1.0),
    Ridge(alpha=0.5),
    ElasticNet(alpha=1.0, l1_ratio=0.5),
    ElasticNet(alpha=1.0, l1_ratio=0.25),
    ElasticNet(alpha=1.0, l1_ratio=0.75),
    ElasticNet(alpha=0.5, l1_ratio=0.5),
    ElasticNet(alpha=0.5, l1_ratio=0.25),
    ElasticNet(alpha=0.5, l1_ratio=0.75),
    KNeighborsRegressor(n_neighbors=5),
    KNeighborsRegressor(n_neighbors=10),
    KNeighborsRegressor(n_neighbors=15),
    KNeighborsRegressor(n_neighbors=20),
    KNeighborsRegressor(n_neighbors=25),
    SVR(kernel='linear'),
    SVR(kernel='poly'),
    SVR(kernel='rbf'),
    SVR(kernel='sigmoid'),
    DecisionTreeRegressor(criterion='squared_error'),
    DecisionTreeRegressor(criterion='friedman_mse'),
    DecisionTreeRegressor(criterion='absolute_error')
]

In [40]:
# Выбор моделей для задания
n = 4
random.seed(my_seed)
my_models1 = random.sample(r_models, n)
print(my_models1)

[LinearRegression(), ElasticNet(alpha=0.5, l1_ratio=0.75), KNeighborsRegressor(n_neighbors=10), SVR(kernel='poly')]


In [70]:
# Загрузим данные для задачи регрессии
data = pd.read_csv("datasets/weather.csv", decimal=',')
data

,water_level,precipitation,temperature,humidity,visibility,wind,weather,pressure,fire,wl_change,temp_change,pressure_change
0,468.0,0.0,24.9,31.0,100.0,4.0,0.0,993.8,4.0,-3.0,-0.3,-0.4
1,140.0,0.0,27.9,22.0,20.0,1.0,0.0,1004.1,4.0,-2.0,0.6,-1.0
2,464.0,0.0,25.6,28.0,20.0,2.0,1.0,1001.2,4.0,-7.0,-1.8,-0.3
3,467.0,0.0,26.7,26.0,100.0,3.0,0.0,992.3,4.0,-1.0,1.2,-1.3
4,138.0,0.0,29.1,22.0,20.0,1.0,0.0,1001.3,4.0,-2.0,2.5,-1.3
...,...,...,...,...,...,...,...,...,...,...,...,...
1434,474.0,0.1,6.4,95.0,4.0,1.0,3.0,999.9,4.0,-1.0,5.9,1.5
1435,65.0,0.0,6.4,93.0,10.0,4.0,3.0,983.9,4.0,-1.0,5.2,3.1
1436,159.0,0.1,8.5,83.0,10.0,2.0,1.0,969.7,4.0,1.0,1.4,1.9
1437,99.0,0.1,7.5,91.0,10.0,3.0,3.0,995.4,4.0,-4.0,2.7,1.7


In [74]:
# Зависимая переменная для всех одна и та же. Предикторы выбираем случайнм образом.
columns = list(data.columns)
n_x = 5

y_label = 'water_level'
x_labels = random.sample(columns[1:], n_x)

print(x_labels)

['fire', 'weather', 'pressure', 'pressure_change', 'humidity']


In [75]:
# Преобразуйте значения всех необходимых параметров к отрезку [0,1].
# Решите получившуюся задачу регрессии с помощью выбранных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

data = data.drop(['precipitation','temperature','visibility','wind','wl_change','temp_change'],axis=1)

In [76]:
data

,water_level,humidity,weather,pressure,fire,pressure_change
0,468.0,31.0,0.0,993.8,4.0,-0.4
1,140.0,22.0,0.0,1004.1,4.0,-1.0
2,464.0,28.0,1.0,1001.2,4.0,-0.3
3,467.0,26.0,0.0,992.3,4.0,-1.3
4,138.0,22.0,0.0,1001.3,4.0,-1.3
...,...,...,...,...,...,...
1434,474.0,95.0,3.0,999.9,4.0,1.5
1435,65.0,93.0,3.0,983.9,4.0,3.1
1436,159.0,83.0,1.0,969.7,4.0,1.9
1437,99.0,91.0,3.0,995.4,4.0,1.7


In [77]:
data.shape
val_test_size = round(0.2*len(data))
print(val_test_size)

288


In [78]:
random_state = my_seed
train_val, test = train_test_split(data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

863 288 288


In [79]:
train.head()

,water_level,humidity,weather,pressure,fire,pressure_change
1437,99.0,91.0,3.0,995.4,4.0,1.7
1006,517.0,72.0,1.0,982.4,4.0,-0.8
527,218.0,40.0,2.0,1025.5,4.0,-0.2
47,179.0,43.0,0.0,1006.2,4.0,-0.6
736,361.0,19.0,0.0,989.2,4.0,-2.5


In [81]:
columns_to_scale = ['fire', 'weather', 'pressure', 'pressure_change', 'humidity']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['fire', 'weather', 'pressure',
                                  'pressure_change', 'humidity'])])

In [82]:
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))
column_names = columns_to_scale + ['water_level']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names
sc_val

,fire,weather,pressure,pressure_change,humidity,water_level
0,0.0,0.500,0.715672,0.655556,0.197802,320.0
1,0.0,0.000,0.762687,0.255556,0.659341,855.0
2,0.0,0.375,0.759701,0.322222,0.846154,515.0
3,0.0,0.125,0.873881,0.344444,0.571429,182.0
4,0.0,0.250,0.720896,0.333333,0.736264,338.0
...,...,...,...,...,...,...
283,0.0,0.750,0.908955,0.477778,0.967033,177.0
284,0.0,0.125,0.560448,0.277778,0.516484,331.0
285,0.0,0.250,0.624627,0.288889,0.285714,127.0
286,0.0,0.125,0.492537,0.277778,0.670330,68.0


In [83]:
r_models = []

r_models.append(ElasticNet(alpha=0.5))
r_models.append(ElasticNet(alpha=0.5, l1_ratio=0.75))

r_models.append(SVR())

r_models.append(Ridge())
r_models

[ElasticNet(alpha=0.5), ElasticNet(alpha=0.5, l1_ratio=0.75), SVR(), Ridge()]

In [84]:
x_labels = column_names[0:-1]
y_labels = ['water_level']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [85]:
for model in r_models:
    print(model)
    model.fit(x_train, y_train)

ElasticNet(alpha=0.5)
ElasticNet(alpha=0.5, l1_ratio=0.75)
SVR()
Ridge()


In [86]:
mses = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(model, '\t', mse)

ElasticNet(alpha=0.5) 	 45384.19143352922
ElasticNet(alpha=0.5, l1_ratio=0.75) 	 45108.5308260631
SVR() 	 48247.23709845016
Ridge() 	 45314.79792875909


In [88]:
i_min = mses.index(min(mses))
best_r_model = r_models[i_min]
best_r_model

ElasticNet(alpha=0.5, l1_ratio=0.75)

In [89]:
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

50282.10272523274


In [90]:
# Задание №2 - анализ моделей для задачи классификации
# Общий список моделей
c_models = [
    LogisticRegression(penalty='none', solver='saga'),
    LogisticRegression(penalty='l1', solver='saga'),
    LogisticRegression(penalty='l2', solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.25, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.75, solver='saga'),
    LogisticRegression(),
    MultinomialNB(alpha=0.0),
    MultinomialNB(alpha=0.25),
    MultinomialNB(alpha=0.5),
    MultinomialNB(alpha=0.75),
    MultinomialNB(alpha=1.0),
    KNeighborsClassifier(n_neighbors=5),
    KNeighborsClassifier(n_neighbors=10),
    KNeighborsClassifier(n_neighbors=15),
    KNeighborsClassifier(n_neighbors=20),
    KNeighborsClassifier(n_neighbors=25),
    SVC(kernel='linear'),
    SVC(kernel='poly'),
    SVC(kernel='rbf'),
    SVC(kernel='sigmoid'),
    DecisionTreeClassifier(criterion='gini', splitter='best'),
    DecisionTreeClassifier(criterion='gini', splitter='random'),
    DecisionTreeClassifier(criterion='entropy', splitter='best'),
    DecisionTreeClassifier(criterion='entropy', splitter='random')
]

In [91]:
# Выбор моделей для задания
n = 5
my_models2 = random.sample(c_models, n)
print(my_models2)

[SVC(kernel='sigmoid'), SVC(kernel='linear'), KNeighborsClassifier(n_neighbors=10), DecisionTreeClassifier(), KNeighborsClassifier(n_neighbors=25)]


In [93]:
# Загрузим данные для задачи классификации
newdata = pd.read_csv("datasets/zoo2.csv")
newdata

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,turtle,0,0,1,0,0,1,0,0,1,1,0,0,4,1,1,1,3
1,chameleon,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
2,iguana,0,0,1,0,0,0,1,1,1,1,0,0,4,1,1,1,3
3,lizard,0,0,1,0,0,0,1,1,1,1,0,0,4,1,0,0,3
4,gecko,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
5,python,0,0,1,0,0,0,1,1,1,1,1,0,0,1,0,1,3
6,boa,0,0,1,0,0,0,1,1,1,1,0,0,0,1,0,1,3
7,adder,0,0,1,0,0,0,1,1,1,1,1,0,0,1,0,1,3
8,crocodile,0,0,1,0,0,1,1,1,1,1,0,0,4,1,0,1,3
9,alligator,0,0,1,0,0,1,1,1,1,1,0,0,4,1,0,1,3


In [94]:
# Метка класса для всех одна и та же. Параметры выбираем случайнм образом.
columns = list(data.columns)
n_x = 8

y_newlabel = 'class_type'
x_newlabels = random.sample(columns[1:-1], n_x)

print(x_newlabels)

['catsize', 'breathes', 'milk', 'tail', 'toothed', 'backbone', 'aquatic', 'predator']


In [95]:
labels = x_newlabels.copy()
labels.append(y_newlabel)
temp_data = newdata[labels]
sec_data = temp_data

In [96]:
sec_data

,catsize,breathes,milk,tail,toothed,backbone,aquatic,predator,class_type
0,1,1,0,1,0,1,1,0,3
1,0,1,0,1,1,1,0,0,3
2,1,1,0,1,1,1,0,1,3
3,0,1,0,1,1,1,0,1,3
4,0,1,0,1,1,1,0,0,3
5,1,1,0,1,1,1,0,1,3
6,1,1,0,1,1,1,0,1,3
7,1,1,0,1,1,1,0,1,3
8,1,1,0,1,1,1,1,1,3
9,1,1,0,1,1,1,1,1,3


In [99]:
# Преобразуйте значения всех необходимых параметров к отрезку [0,1].
# Решите получившуюся задачу классификации с помощью выбранных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

sec_data.shape
val_test_size2 = round(0.2*len(sec_data))
print(val_test_size2)

9


In [101]:
random_state = my_seed
train_val2, test2 = train_test_split(sec_data, test_size=val_test_size2, random_state=random_state)
sec_train, val2 = train_test_split(train_val2, test_size=val_test_size2, random_state=random_state)
print(len(train2), len(val2), len(test2))

25 9 9


In [102]:
sec_train.head()

,catsize,breathes,milk,tail,toothed,backbone,aquatic,predator,class_type
13,1,0,0,1,0,1,1,0,4
41,1,0,0,0,0,0,1,0,7
23,0,1,0,0,1,1,1,0,5
33,0,1,0,0,0,0,0,0,6
34,0,1,0,0,0,0,0,0,6


In [103]:
columns_to_scale2 = x_newlabels

ct2 = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale2)], remainder='passthrough')
ct2.fit(sec_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['catsize', 'breathes', 'milk', 'tail',
                                  'toothed', 'backbone', 'aquatic',
                                  'predator'])])

In [105]:
sc_train2 = pd.DataFrame(ct2.transform(sec_train))
sc_test2 = pd.DataFrame(ct2.transform(test2))
sc_val2 = pd.DataFrame(ct2.transform(val2))
column_names2 = columns_to_scale2 + ['class_type']
sc_train2.columns = column_names2
sc_test2.columns = column_names2
sc_val2.columns = column_names2
sc_val2

,catsize,breathes,milk,tail,toothed,backbone,aquatic,predator,class_type
0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,5.0
1,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,3.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
3,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,3.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
5,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,3.0
6,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,5.0
7,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0
8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0


In [107]:
# Вспоминаем алгоритмы решения задачи классификации
c_models = []

# Логистическая регрессия
# Параметр penalty - тип регуляризации: 'l1', 'l2', 'elasticnet', 'none'}, по умолчанию 'l2'
# Для некоторых типов регуляризации доступны не все алгоритмы (параметр solver)
# Для elasticnet регуляризации необходимо уазывать параметр l1_ratio (0 - l2, 1 - l1)
c_models.append(LogisticRegression(penalty='l1', solver='saga'))


# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
c_models.append(KNeighborsClassifier(n_neighbors=5))

# Метод опорных векторов
# Параметр kernel опредеяет вид ядра преобразования
c_models.append(SVC(kernel='sigmoid'))

# Деревья решений
# Параметр criterion - критерий качества ветвления: 'gini', 'entropy'
# Параметр splitter - стартегия ветвления: 'best', 'random'
c_models.append(DecisionTreeClassifier())
c_models.append(DecisionTreeClassifier(splitter='random'))
c_models

[LogisticRegression(penalty='l1', solver='saga'),
 KNeighborsClassifier(),
 SVC(kernel='sigmoid'),
 DecisionTreeClassifier(),
 DecisionTreeClassifier(splitter='random')]

In [109]:
x_labels2 = column_names2[0:-1]
y_labels2 = ['class_type']

x_train2 = sc_train2[x_labels2]
x_test2 = sc_test2[x_labels2]
x_val2 = sc_val2[x_labels2]

y_train2 = np.ravel(sc_train2[y_labels2])
y_test2 = np.ravel(sc_test2[y_labels2])
y_val2 = np.ravel(sc_val2[y_labels2])
x_train2

,catsize,breathes,milk,tail,toothed,backbone,aquatic,predator
0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
6,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
9,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [110]:
# Обучаем модели
for model2 in c_models:
    print(model2)
    model2.fit(x_train2, y_train2)

LogisticRegression(penalty='l1', solver='saga')
KNeighborsClassifier()
SVC(kernel='sigmoid')
DecisionTreeClassifier()
DecisionTreeClassifier(splitter='random')


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


In [111]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model2 in c_models:
    val_pred2 = model2.predict(x_val2)
    f1 = f1_score(y_val2, val_pred2, average='weighted')
    f1s.append(f1)
    print(model2, '\t', f1)

LogisticRegression(penalty='l1', solver='saga') 	 0.42777777777777776
KNeighborsClassifier() 	 0.39814814814814814
SVC(kernel='sigmoid') 	 0.42777777777777776
DecisionTreeClassifier() 	 0.5759259259259258
DecisionTreeClassifier(splitter='random') 	 0.5759259259259258


In [112]:
# Выбираем лучшую модель
i_max = f1s.index(max(f1s))
best_c_model = c_models[i_max]
best_c_model

DecisionTreeClassifier()

In [113]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred2 = best_c_model.predict(x_test2)
f1 = f1_score(y_test2, test_pred2, average='weighted')
print(f1)

0.8814814814814814


In [144]:
# Вспоминаем алгоритм решения задачи кластеризации - метод k-средних
# Параметр n_clusters - количество кластеров, по умолчанию равен 8
k_models = []
k_models.append(KMeans(n_clusters=5))
k_models.append(KMeans(n_clusters=8))
k_models.append(KMeans(n_clusters=20))
k_models.append(KMeans(n_clusters=40))

In [145]:
# Выделим используемые параметры
x_labels2 = column_names2[0:-1]
x_data2 = pd.concat([sc_train2[x_labels2], sc_val2[x_labels2], sc_test2[x_labels2]])
x_data2

,catsize,breathes,milk,tail,toothed,backbone,aquatic,predator
0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
6,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
9,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [149]:
# Произведем кластеризацию
for model2 in k_models:
    model2.fit(x_data2)

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:3: ConvergenceWarning: Number of distinct clusters (15) found smaller than n_clusters (20). Possibly due to duplicate points in X.
  This is separate from the ipykernel package so we can avoid doing imports until
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:3: ConvergenceWarning: Number of distinct clusters (15) found smaller than n_clusters (40). Possibly due to duplicate points in X.
  This is separate from the ipykernel package so we can avoid doing imports until


In [150]:
# Оценим качество результата
sils = []
for model2 in k_models:
    cluster_labels = model2.predict(x_data2)
    s = silhouette_score(x_data2, cluster_labels)
    sils.append(s)
    print(s)

0.5954276125075004
0.7314485401295943
0.8604651162790697
0.8604651162790697


In [151]:
# Выбираем лучшую модель
i_max = sils.index(max(sils))
best_k_model = k_models[i_max]
print(best_k_model)
print(sils[i_max])

KMeans(n_clusters=20)
0.8604651162790697
